# Problems with 3D dataset rendering

## One by one loading using Aicsimage

In [ ]:
import os
from aicsimageio import AICSImage
from aicsimageio.readers import BioformatsReader

In [ ]:
import napari

v = napari.Viewer()

img = r"G:\Stardist_results\EH3574_Oct4.tif"
short_name = os.path.splitext(img)

storeim = AICSImage(img, reader=BioformatsReader)
image= storeim.dask_data
storeim.xarray_dask_data
        
v.add_image(image, name='Image '+str(short_name[0]), contrast_limits=[0,10000])
        
print(img+' is in Napari')
print(image.shape, image.size, image.dtype)

In [ ]:
lbl = r"G:\Stardist_results\EH3574_Oct4_lb.tif"
short_namel = os.path.splitext(lbl)

storelb = AICSImage(lbl, reader=BioformatsReader)
labels = storelb.dask_data
storelb.xarray_dask_data
        
v.add_labels(labels, name='label '+str(short_namel[0]))
        
print(lbl+' is in Napari')
print(labels.shape, labels.size, labels.dtype)

## Load with dask-image

In [ ]:
import napari
from dask_image.imread import imread

In [ ]:
v = napari.Viewer()
img = imread(r"G:\Stardist_results\EH3574_Oct4.tif")
v.add_image(img, contrast_limits=[0,10000], multiscale=False)
print(img+' is in Napari')
print(image.shape, image.size, image.dtype)

In [ ]:
lbl = imread(r"G:\Stardist_results\EH3574_Oct4_lb.tif")
v.add_labels(lbl, multiscale=False)
print(lbl+' is in Napari')
print(labels.shape, labels.size, labels.dtype)

## Convert to ngff and load in Napari

In [1]:
import napari
import os
import zarr
import tifffile as tf
from ome_zarr.writer import write_image as writeimg
from ome_zarr.writer import write_labels as writelbl
from ome_zarr.io import parse_url

In [ ]:
path = r"G:\Stardist_results\test_ngff_image"

img = tf.imread(r'G:\Stardist_results\EH3574_Oct4.tif')
store = parse_url(path, mode="w").store
root = zarr.group(store=store)

writeimg(image=img, group=root, axes="zyx", storage_options=dict(chunks=(1, 7177, 4117)))

#print(img+' is in Napari')
print(img.shape, img.size, img.dtype)
#

In [2]:
path2 = r"G:\Stardist_results\test_ngff_labels"

lbl = tf.imread(r'G:\Stardist_results\EH3574_Oct4_lb.tif')
store = parse_url(path2, mode="w").store
root = zarr.group(store=store)

writeimg(image=lbl, group=root, axes="zyx", storage_options=dict(chunks=(1, 7177, 4117)))

#print(img+' is in Napari')
print(lbl.shape, lbl.size, lbl.dtype)

(1053, 7177, 4117) 31113737577 int32


In [ ]:
def convert_label_data(in_path, group, resolution, units, label_name, colors=None):
    # load the input data from ome.tif
    vol = tf.imread(r'G:\Stardist_results\EH3574_Oct4_lb.tif')
    if vol.ndim != 3:
        print("Labels have unexpected shape", vol.shape, "for", in_path, label_name)
        print("Adding these labels will be skipped")
        return

    # create scale pyramid
    scaler = ome_zarr.scale.Scaler()
    mip = scaler.nearest(vol)

    # specify the axis and transformation metadata
    axis_names = tuple("zyx")
    axes, trafos = get_axes_and_trafos(mip, axis_names, units, resolution)

    # provide additional storage options for zarr
    storage_opts = get_storage_opts(axis_names)

    ome_zarr.writer.write_multiscale_labels(
        mip, group, label_name,
        axes=axes, coordinate_transformations=trafos,
        storage_options=storage_opts, colors=colors
    )

In [ ]:
lbl = tf.imread(r'G:\Stardist_results\EH3574_Oct4_lb.tif')
scaler = ome_zarr.scale.Scaler()
mip = scaler.nearest(lbl)